In [ ]:
# Version 1 - original from Facebook 資料科學小嫩嫩 的派大 Patrick Juan

In [23]:
import os, sys

sys.path.append('.')

from langchain.llms import Ollama

model="llama2:70b"
client = Ollama(base_url="http://localhost:11434", model=model)

In [24]:
# pip3 install gradio pypdf chromadb
# ollama pull nomic-embed-text

In [25]:
import gradio as gr
#import ollama
#from bs4 import BeautifulSoup as bs
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
'''
# Load the data from the web URL
url = 'https://en.wikipedia.org/wiki/Ohiya'
loader = WebBaseLoader(url)
docs = loader.load()
'''
from langchain_community.document_loaders import PyPDFLoader
file_path = './50_Years_of_AI.pdf'
loader = PyPDFLoader(file_path)
docs = []
async for page in loader.alazy_load():
    docs.append(page)


In [26]:
# Split the loaded documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [27]:
# Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)


In [30]:
# Define the function to call the Ollama Llama3 model
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
#     response = client.chat.completions.create(model=model, messages=[{'role': 'system', 'content': '總是用繁體中文回答！'}, {'role': 'user', 'content': formatted_prompt}])
    response = Ollama.chat(
        model=model,  # Specify the model name
        messages=[
            {'role': 'system', 'content': '總是用繁體中文回答！'},
            {'role': 'user', 'content': formatted_prompt}
        ]
    )
#    return response.choices[0].message.content
    return response['content']


# Define the RAG setup
retriever = vectorstore.as_retriever()

def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return ollama_llm(question, formatted_context)

# Define the Gradio interface
def get_important_facts(question):
    return rag_chain(question)

# Create a Gradio app interface
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="Please summarize in 500 words"),
  outputs="text",
  title="RAG with Llama3",
  description="Ask questions about the proveded context",
)

# Launch the Gradio app
iface.launch()


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [31]:
import chromadb
print("chromadb installed and loaded successfully")


chromadb installed and loaded successfully
